<a href="https://colab.research.google.com/github/nasebah/arabic-text-summarization/blob/main/summ_task_Freq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
# Source: https://github.com/RaghadAlshaikh/Automatic-Arabic-Text-Summarizer/tree/master

from urllib import request
from bs4 import BeautifulSoup as bs
import re
import nltk
from nltk.corpus import stopwords
import heapq
from nltk.stem.isri import ISRIStemmer
nltk.download('punkt')
nltk.download('stopwords')


#Store the article URL
#MSA URL
url1 = "https://ar.wikipedia.org/wiki/%D9%84%D9%87%D8%AC%D8%A9_%D9%85%D8%B5%D8%B1%D9%8A%D8%A9"
#Egyptian Arabic URL
#url1= "https://arz.wikipedia.org/wiki/%D8%A7%D9%84%D9%84%D8%BA%D9%87_%D8%A7%D9%84%D9%85%D8%B5%D8%B1%D9%8A%D9%87_%D8%A7%D9%84%D8%AD%D8%AF%D9%8A%D8%AB%D9%87"
allparagraphContent1 = ""
#Access the articles URL
htmlArticle1 = request.urlopen(url1)
#Get the articles code including all the HTML tags
Soup1 = bs(htmlArticle1, 'html.parser')
paragraphContents1 = Soup1.findAll('p')

#Get the text
for paragraphContent in paragraphContents1:
    allparagraphContent1 += paragraphContent.text

# preprocessing by cleaning text, removing stop words, and applying lemmatization. I will write a function and apply it to the whole data set. mee
# Now I shall apply the function I wrote on the whole dataset and store the result in a new column named “text_clean” so that you can choose to work with the raw corpus or the preprocessed text.
# ====== Clean input by removing all the HTML tags information======
allparagraphContent_Cleaned1 = re.sub(r'\[0-9]*\]',' ',allparagraphContent1)
allparagraphContent_Cleaned1 = re.sub(r'\s+',' ',allparagraphContent1)
allparagraphContent_Cleaned1 = re.sub(r'\[^a-zA-Z]',' ',allparagraphContent1)
allparagraphContent_Cleaned1 = re.sub(r'\s+',' ',allparagraphContent1)

# Sentence Splitting   (convert from string to list) me)
sentences_tokens1 = nltk.sent_tokenize(allparagraphContent_Cleaned1)
print(sentences_tokens1) #me
# Tokenization
words_tokens1 = nltk.word_tokenize(allparagraphContent_Cleaned1)
print("\n \n " , words_tokens1) #me
# Arabic StopWords List
stopwords_list1 = stopwords.words('arabic')
# That function removes a set of words from the corpus if given.
# I can create a list of generic stop words for the English vocabulary with nltk (we could edit this list by adding or removing words).
print(stopwords_list1 )
# Arabic Stemming   (convert the word into root word) me
st = ISRIStemmer()
words_stemm1 = [st.stem(word) for word in words_tokens1]
#
# Calculate Each Term Frequency ,  table Frequency
word_frequencies1 = {} # Dictionary
for word in words_stemm1:
    if word not in stopwords_list1: #if its not stopword  so we dont count it in word freq
        if word not in word_frequencies1.keys(): # if it show first time
            word_frequencies1[word] = 1
        else:  # if it show more than one time
            word_frequencies1[word] += 1
#Get MAX
maximum_frequency_word1 = max(word_frequencies1.values())
print("max", maximum_frequency_word1) # me , haight word frequinted
print(word_frequencies1) # me
#Normalize
for word in word_frequencies1.keys(): # loop for the keys not the values
    word_frequencies1[word] = (word_frequencies1[word]/maximum_frequency_word1) # we will devide each value in the dictionary by 36 the max
print(word_frequencies1) #normlize me
# Calculate Each Sentence Frequency from the Term Frequencies
#Calculate Sentence scores based on each word within sentence
sentences_scores1 = {} # Dictionary
wordsCounter = 0.0
for sentence in sentences_tokens1:
    for word in nltk.word_tokenize(sentence):
        word = st.stem(word)
        if word in word_frequencies1.keys():
            wordsCounter += 1;
            if sentence not in sentences_scores1.keys():
                sentences_scores1[sentence] = word_frequencies1[word]  # كل كلمة ياخذ قيمة النورملايز ويزيده عشان نشوف من الجملة الاكثر اهمية
            else:
                sentences_scores1[sentence] += word_frequencies1[word] # diffrent add + , which add the previous

#Normalize
    sentences_scores1[sentence] = sentences_scores1[sentence]/wordsCounter
    #Reset Counter
    wordsCounter = 0

print(sentences_scores1)

#Get summary with only highest top 3
print("Number of tokens for article in Modern Standard Arabic: ",len(words_tokens1))
print("Number of sentences for article in Modern Standard Arabic: ",len(sentences_tokens1))
summary = heapq.nlargest(3, sentences_scores1, key=sentences_scores1.get)
# Print the summarized text
print("Predicted Summary of the article in Modern Standard Arabic:\n", summary)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[' اللغه المصريه الحديثه (زى ما سماها بيومى قنديل)[1] او المصرى هى اللغه اللى المصريين بيتكلّموها فى مصر,[2][3][4] ابتدا تاريخها فى دلتا النيل حوالين بلادها الحضريه زى القاهره واسكندريه.', '[2] النهارده اللغه المصرى هى اللغه السايده فى مصر وبتتكون من لهجات زى مصرى قاهراوى ، بحيرى او مصرى دلتاوى.', 'فى جنوب مصر الناس بتتكلم مصري صعيدى و هى لغه بنفسيها معترف بيها فى العالم عن طريق كود اللغه بالنسبه لقانون المنظمه الدوليه للتوحيد القياسى (ايزو 3-639), و هو aec.', 'شويه من اهل وجه بحرى و القاهره بيلاقوا صعوبه فى فهم شوية كلام صعيدى.', 'المصرى لغه ماخدتش نصيبها من الكتابه, بس هي لغة جميع الاغاني والافلام والمسلسلات المصريةو ساعات بيتكتب بيها فى الروايات, و معظم المسرحيات, و الاشعار و الازجال و كمان فى مجلات ورسوم الكاركاتير و الاعلانات, و شوية جرانيل, وأخيراً فى كتب انكتبت بـ اللغه المصرى الحديثة.', '[5] أكترية طرق الاعلام المكتوبه و أخبار التليڤيزيون بتبقى بـ العربى الكلاسيكى بعد تطويرهُ و تعديلهُ.', 'دلوقتى بقى فيه شويِة قنوات بتستعمل "المصرى" بدل "العربى فى الاخبار, زى قناة ”OTV“.', 'الم